<a href="https://colab.research.google.com/github/minerado/ml/blob/main/gpt-implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT Implementation

This implementation is based on Andej Karpathy's tutorial [Let's build GPT](https://www.youtube.com/watch?v=kCc8FmEb1nY)

For this exercise, we'll be using the tinyshakespare data as vocabulary:

In [27]:
# Fetch and save data
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [28]:
with open('input.txt', 'r') as f:
    text = f.read()

In [29]:
print(f"Length of dataset: {len(text)} characters")

Length of dataset: 1115394 characters


In [30]:
# Let's inspect it
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



## Build the vocabulary

In [31]:
# Unique characters to appear in the data
chars = sorted(set(text))
vocab_size = len(chars)

print(f"Vocabulary: {''.join(chars)}")
print(f"Vocabulary size: {vocab_size}")

Vocabulary: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary size: 65


## Encoding/Decoding functions

In [32]:
# Create mapping from characters to integers
char_dict = {char: i for i, char in enumerate(chars)}
index_dict = {i: char for i, char in enumerate(chars)}

def encode(s): return [char_dict[c] for c in s]
def decode(lst): return ''.join([index_dict[i] for i in lst])

In [33]:
decode(encode("hello world"))

'hello world'

## Build data for training

In [34]:
# Let's now encode the entire text and store it into a tensor
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [35]:
# Separate train and validation data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

print(f'Train data shape: {train_data.shape}')
print(f'Validation data shape: {val_data.shape}')

Train data shape: torch.Size([1003854])
Validation data shape: torch.Size([111540])


## Uderstanding block size

In [36]:
## From HuggingFace Docs:
## block_size: This is the maximum sequence length or length of one block of text.
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
  print(f"Input: {x[:i+1].tolist()}; Target: {y[i]}")
print(x, y)

Input: [18]; Target: 47
Input: [18, 47]; Target: 56
Input: [18, 47, 56]; Target: 57
Input: [18, 47, 56, 57]; Target: 58
Input: [18, 47, 56, 57, 58]; Target: 1
Input: [18, 47, 56, 57, 58, 1]; Target: 15
Input: [18, 47, 56, 57, 58, 1, 15]; Target: 47
Input: [18, 47, 56, 57, 58, 1, 15, 47]; Target: 58
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [37]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(data):
  random_idxs = torch.randint(len(train_data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in random_idxs])
  y = torch.stack([data[i+1:i+block_size+1] for i in random_idxs])

  return x, y

xb, yb = get_batch(train_data)

print(f"Input shape: {xb.shape}")
print(f"Input: {xb}")
print(f"Target shape: {yb.shape}")
print(f"Target: {yb}")

Input shape: torch.Size([4, 8])
Input: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Target shape: torch.Size([4, 8])
Target: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [80]:
import torch
from torch import nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, i, target):
    logits = self.embedding_table(i)

    loss = F.cross_entropy(logits.permute(0, 2, 1), target)

    return logits, loss

In [81]:
bm = BigramLanguageModel(vocab_size)
out, loss = bm(xb, yb)
print(out.shape)
print(loss)

torch.Size([4, 8, 65])
tensor(4.8786, grad_fn=<NllLoss2DBackward0>)


In [82]:
# I'm stopping to study more about generation